In [1]:
import CompFlowCalcs as CF
import basically_ti_basic.tokens as ti_tokens
import inspect

In [2]:
classes = [name for name, obj in inspect.getmembers(CF) if inspect.isclass(obj)]
print(classes)


['ExpansionWaveRatio', 'IsentropicRatio', 'NormalShockRatio', 'ObliqueShockResult', 'PressureInverseIsentropicRatio']


In [3]:
import inspect

init_contents = {}
for class_name in classes:
  class_obj = getattr(CF, class_name)
  init_func = getattr(class_obj, '__init__')
  init_source = inspect.getsource(init_func)
  init_contents[class_name] = init_source

init_contents


{'ExpansionWaveRatio': '    def __init__(self, gamma, mach):\n        self.velocity_ratio = 1 / (1 + (gamma - 1) / 2 * mach)\n        self.pressure_ratio = 1 / (1 + (gamma - 1) / 2 * mach) ** (2 * gamma / (gamma - 1))\n        self.temp_ratio = 1 / (1 + (gamma - 1) / 2 * mach) ** 2\n        self.density_ratio = 1 / (1 + (gamma - 1) / 2 * mach) ** (2 / (gamma - 1))\n',
 'IsentropicRatio': '    def __init__(self, gamma, mach):\n        self.temp_ratio = 1 + (gamma - 1) / 2 * mach ** 2\n\n        self.pressure_ratio = self.temp_ratio ** (gamma / (gamma - 1))\n        self.density_ratio = self.temp_ratio ** (1 / (gamma - 1))\n',
 'NormalShockRatio': '    def __init__(self, gamma, mach):\n        self.pressure_ratio = (2 * gamma * mach ** 2 - (gamma - 1)) / (gamma + 1)\n        self.temp_ratio = (2 * gamma * mach ** 2 - (gamma - 1)) * ((gamma - 1) * mach ** 2 + 2) / ((gamma + 1) ** 2 * mach ** 2)\n        self.density_ratio = (gamma + 1) * mach ** 2 / ((gamma - 1) * mach ** 2 + 2)\n\n      

In [4]:
init_contents[classes[0]]

'    def __init__(self, gamma, mach):\n        self.velocity_ratio = 1 / (1 + (gamma - 1) / 2 * mach)\n        self.pressure_ratio = 1 / (1 + (gamma - 1) / 2 * mach) ** (2 * gamma / (gamma - 1))\n        self.temp_ratio = 1 / (1 + (gamma - 1) / 2 * mach) ** 2\n        self.density_ratio = 1 / (1 + (gamma - 1) / 2 * mach) ** (2 / (gamma - 1))\n'

In [5]:
import inspect

symbols_per_class = {}
for class_name in classes:
    class_obj = getattr(CF, class_name)
    init_func = getattr(class_obj, '__init__')
    init_source = inspect.getsource(init_func)
    
    # Extract symbols from the init function source code
    symbols = [symbol.split('=')[0].strip() for symbol in init_source.split('\n') if '=' in symbol]
    
    symbols_per_class[class_name] = symbols

symbols_per_class

{'ExpansionWaveRatio': ['self.velocity_ratio',
  'self.pressure_ratio',
  'self.temp_ratio',
  'self.density_ratio'],
 'IsentropicRatio': ['self.temp_ratio',
  'self.pressure_ratio',
  'self.density_ratio'],
 'NormalShockRatio': ['self.pressure_ratio',
  'self.temp_ratio',
  'self.density_ratio',
  'self.stag_ratio',
  'stag_one',
  'self.exit_stag_ratio',
  'self.mach_two'],
 'ObliqueShockResult': ['self.theta',
  'mach_one_normal',
  'self.shock_ratio: NormalShockRatio',
  'mach_two_normal',
  'self.mach_two'],
 'PressureInverseIsentropicRatio': ['self.temp_ratio', 'self.mach']}

In [6]:
all_symbols = set()
for symbols in symbols_per_class.values():
    all_symbols.update(symbols)

all_symbols


{'mach_one_normal',
 'mach_two_normal',
 'self.density_ratio',
 'self.exit_stag_ratio',
 'self.mach',
 'self.mach_two',
 'self.pressure_ratio',
 'self.shock_ratio: NormalShockRatio',
 'self.stag_ratio',
 'self.temp_ratio',
 'self.theta',
 'self.velocity_ratio',
 'stag_one'}

In [7]:
total_symbols = len(all_symbols)
total_symbols


13

In [8]:
init_parameters = {}
for class_name, init_source in init_contents.items():
    init_func = getattr(CF, class_name).__init__
    init_signature = inspect.signature(init_func)
    parameters = list(init_signature.parameters.keys())[1:]  # Exclude the 'self' parameter
    init_parameters[class_name] = parameters

function_parameters = set()
for parameters in init_parameters.values():
    function_parameters.update(parameters)

function_parameters


{'beta', 'gamma', 'mach', 'mach_one', 'pressure_ratio'}

In [9]:
all_symbols

{'mach_one_normal',
 'mach_two_normal',
 'self.density_ratio',
 'self.exit_stag_ratio',
 'self.mach',
 'self.mach_two',
 'self.pressure_ratio',
 'self.shock_ratio: NormalShockRatio',
 'self.stag_ratio',
 'self.temp_ratio',
 'self.theta',
 'self.velocity_ratio',
 'stag_one'}

In [13]:
inv = ti_tokens.get_inverse_tokens()
# inv

In [15]:
symbol_token_map = {
  # Output Tokens
  'self.density_ratio': 'D',
  'self.pressure_ratio': 'P',
  'self.temp_ratio': 'T',
  'self.theta': '[theta]',
  'self.exit_stag_ratio': 'E',
  'self.stag_ratio': 'S',
  'self.velocity_ratio': 'V',
  'self.mach_two': '?',
  
  # Input Tokens
  'beta': 'B',
  'gamma': 'G',
  'mach': 'M',
}

tokens = set(symbol_token_map.values())
no_duplicate_tokens = len(tokens) == len(symbol_token_map)
assert no_duplicate_tokens


values_in_map = set(symbol_token_map.values())
keys_in_inv = set(inv.keys())

all_values_in_inv = values_in_map.issubset(keys_in_inv)
assert(all_values_in_inv)


symbols_not_in_map = [symbol for symbol in all_symbols if symbol not in symbol_token_map]
symbols_not_in_map
# assert len(symbols_not_in_map) == 0

['mach_two_normal',
 'self.shock_ratio: NormalShockRatio',
 'self.mach',
 'stag_one',
 'mach_one_normal']

In [16]:
translated_init_contents = {}
for class_name, init_source in init_contents.items():
    translated_source = init_source
    for symbol, token in symbol_token_map.items():
        translated_source = translated_source.replace(symbol, token)
    translated_init_contents[class_name] = translated_source
translated_init_contents


{'ExpansionWaveRatio': '    def __init__(self, G, M):\n        V = 1 / (1 + (G - 1) / 2 * M)\n        P = 1 / (1 + (G - 1) / 2 * M) ** (2 * G / (G - 1))\n        T = 1 / (1 + (G - 1) / 2 * M) ** 2\n        D = 1 / (1 + (G - 1) / 2 * M) ** (2 / (G - 1))\n',
 'IsentropicRatio': '    def __init__(self, G, M):\n        T = 1 + (G - 1) / 2 * M ** 2\n\n        P = T ** (G / (G - 1))\n        D = T ** (1 / (G - 1))\n',
 'NormalShockRatio': '    def __init__(self, G, M):\n        P = (2 * G * M ** 2 - (G - 1)) / (G + 1)\n        T = (2 * G * M ** 2 - (G - 1)) * ((G - 1) * M ** 2 + 2) / ((G + 1) ** 2 * M ** 2)\n        D = (G + 1) * M ** 2 / ((G - 1) * M ** 2 + 2)\n\n        S = T ** (-G / (G - 1)) * P\n\n        stag_one = IsentropicRatio(G, M).pressure_ratio\n        E = S * stag_one\n        ? = ((1 + (G - 1) / 2 * M ** 2) / (G * M ** 2 - (G - 1) / 2)) ** 0.5\n',
 'ObliqueShockResult': '    def __init__(self, G, M_one, B):\n        [theta] = math.atan(2 / math.tan(B) * (M_one ** 2 * math.sin

In [17]:
split_init_contents = {}
for class_name, init_source in translated_init_contents.items():
  l = [line.lstrip() for line in init_source.splitlines()[1:] if '=' in line]

  l = [line.replace("**", "^") for line in l]
  

  split_init_contents[class_name] = []
  for line in l:
    s = line.split('=')
    split_init_contents[class_name].append(f"{s[1].strip()} → {s[0].strip()}".strip())

split_init_contents


{'ExpansionWaveRatio': ['1 / (1 + (G - 1) / 2 * M) → V',
  '1 / (1 + (G - 1) / 2 * M) ^ (2 * G / (G - 1)) → P',
  '1 / (1 + (G - 1) / 2 * M) ^ 2 → T',
  '1 / (1 + (G - 1) / 2 * M) ^ (2 / (G - 1)) → D'],
 'IsentropicRatio': ['1 + (G - 1) / 2 * M ^ 2 → T',
  'T ^ (G / (G - 1)) → P',
  'T ^ (1 / (G - 1)) → D'],
 'NormalShockRatio': ['(2 * G * M ^ 2 - (G - 1)) / (G + 1) → P',
  '(2 * G * M ^ 2 - (G - 1)) * ((G - 1) * M ^ 2 + 2) / ((G + 1) ^ 2 * M ^ 2) → T',
  '(G + 1) * M ^ 2 / ((G - 1) * M ^ 2 + 2) → D',
  'T ^ (-G / (G - 1)) * P → S',
  'IsentropicRatio(G, M).pressure_ratio → stag_one',
  'S * stag_one → E',
  '((1 + (G - 1) / 2 * M ^ 2) / (G * M ^ 2 - (G - 1) / 2)) ^ 0.5 → ?'],
 'ObliqueShockResult': ['math.atan(2 / math.tan(B) * (M_one ^ 2 * math.sin(B) ^ 2 - 1) / (M_one ^ 2 * (G + math.cos(2 * B)) + 2)) → [theta]',
  'M_one * math.sin(B) → M_one_normal',
  'NormalShockRatio(G, M_one_normal) → self.shock_ratio: NormalShockRatio',
  'self.shock_ratio.M_two → M_two_normal',
  'M_two_norm

In [18]:
import os

# Create a directory to save the .ti files
os.makedirs("ti_files", exist_ok=True)

# Strip spaces from each value in split_init_contents and save as .ti file
for class_name, contents in split_init_contents.items():
  stripped_contents = [content.replace(" ", "") for content in contents]
  file_path = f"ti_files/{class_name}.ti"
  with open(file_path, "w") as file:
    file.write("\n".join(stripped_contents))
